# Analysis for IDP in replica exchange

Author: Yumeng Zhang
Contact: yumzhang@umass.edu
Date: 2023/07/17

```
This is a tutorial for general analysis for protein conformations with replica exchange method.

We will start from the single protein conformational property studies, then take the replica exchange into consideration. 

Properties include: 
1. 2nd structures.
2. Rg
3. Ree
4. Temperature random walk
...

Package needed: 
See import below.
```

In [1]:
import numpy as np
from numpy import ma

from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.colors import LogNorm
from matplotlib import ticker, cm

import mdtraj as md
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


import scipy.stats as stats
from scipy.stats import gaussian_kde
import scipy.stats as st
from scipy.stats import norm
from scipy.stats import gaussian_kde
import scipy.stats as st

## Conformational property analysis

### 2nd conformational analysis

In [ ]:
# Load your PDBs 
top = md.load_pdb(str(i)+'.pdb')

# Load your trajectory:
traj1=md.load_dcd('prod.1.dcd',top='prot.pdb') # for dcds
traj1=md.load('prod.1.xtc',top='prot.pdb')     # for xtcs

# Compute dssp: 
dssp0 = md.compute_dssp(traj1,simplified=True) # simplified can be "False", to specifically calculate 2nd properties 

h = np.where(dssp0=='H', 1, 0) # Get helicity properties
e = np.where(dssp1=='E', 1, 0) # Get sheet properties

h1=np.sum(h,0) / h.shape[0]   # calculate average, (no need if want to plot by time)
e1=np.sum(e,0) / e.shape[0]   # similar

            
            
plt.plot(np.arange(len(h1)),h1,label='',linewidth=3,capsize=5.5,color='') # plot the average 
plt.plot(np.arange(len(h)),h[i],label='',linewidth=3,capsize=5.5,color='') # plot helicity as a function of Time for residue i
       
        
plt.legend(frameon=False,fontsize=16) # show legend


# define plot range
plt.ylim([0,0.2]) 
plt.xlim([0,25])  

# define labels ... figure size...
plt.ylabel('Probability',fontsize=16)
plt.xlabel('GY23 Residue',fontsize=16)
plt.title('Single Chain Helicity',fontsize=16)
plt.tick_params(axis='both', labelsize=16,direction='in',length=8,width=2)
plt.rcParams['axes.linewidth'] = 2
plt.rcParams["figure.figsize"] = (10,5)

### Rg analysis

In [ ]:
# Load traj
traj1=md.load('../s2-prod/'+str(i)+'.1.dcd',top='../s1-model/'+str(i)+'.pdb')
        
# compute Rg (*10 to change to Angstrom        
rg=md.compute_rg(traj1)[20000:]*10

### calculate and plot average
### ----------------------------------------
rgave=rg.mean()
rgstd=rg.std()        
plt.errorbar(exp[inx],rggave,yerr=rgstd, label=sim[inx],linewidth=3,capsize=5.5,marker=markers[inx])


### Histogram
### ----------------------------------------------------
# calculate and plot histogram and error bars
half=len(rg)/2
# block average
rg1=rg[0:half]
rg2=rg[half:]
print('>>> Check the shape of the data after blocking:', rg.shape,rg1.shape,rg2.shape)

# normalization 
weight1=np.ones_like(rg1)/float(len(rg1))
weight2=np.ones_like(rg2)/float(len(rg2))

# histogram, key point: bins! carefully choose the bin size and bin ranges.
x1=np.histogram(rg1,bins=np.linspace(5,55,50),weights=weight1)
x2=np.histogram(rg2,bins=np.linspace(5,55,50),weights=weight2)

# x1[1]: bins, x1[0]: probability
xdat=(x1[1][0:-1]+x1[1][1:])/2
y1dat=x1[0]
y2dat=x2[0]

rave=np.mean([y1dat,y2dat],axis=0)
rstd=np.std([y1dat,y2dat],axis=0)

# plot
plt.errorbar(xdat,rave,yerr=rstd, label=label2[0],linewidth=3,capsize=5.5,color='b')


plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False,fontsize=12)
plt.ylim([10,50])
plt.xlim([10,50])
        
plt.ylabel('HyRes II (A)',fontsize=16)
plt.xlabel('EXP (A)',fontsize=16)
plt.title('Single Chain Rg',fontsize=16)
plt.tick_params(axis='both', labelsize=16,direction='in',length=8,width=2)
plt.rcParams['axes.linewidth'] = 2
plt.rcParams["figure.figsize"] = (6,6)

### Ree analysis

```
This is a full example I used to set up some loop, variables...
```

In [ ]:
a='wt'
b='h2k'
c='h20k'
d='hk'
sim=a

#temp=[250,260,270,280,290,300,310]
temp=[300]
t=300
# %cd /home/yumzhang/pikes_work/8-llps/gy23-single-chain/s3-prod/


labels=['GY23 (WT)','GY23 (H2/K)','GY23 (H20/K)','GY23 (H/K)']
colors=['black','blue','purple','red']
inx=0
nfig=len(temp)*6
plt.tick_params(axis='both', labelsize=16,direction='in',length=8,width=2)
plt.rcParams['axes.linewidth'] = 2
plt.rcParams["figure.figsize"] = (nfig,5)
index=1
for m in temp:
      
    plt.subplot(1,len(temp),index)
   
   
    %cd /home/yumzhang/pikes_work/8-llps/gy23-hyres-200rep-45/from-lsl/wt/
    top = md.load_pdb('../../s2-npt/'+str(sim)+'.pdb')
       
    traj=md.load(str(sim)+'.'+str(m)+'K.xtc',top='../../s2-npt/'+str(sim)+'.pdb')[-500:] # load last 500 frame
   
    for i in range(0,200):
        start=i*24+i+2
        end=(i+1)*24+i-1
        #print(start,end)
        atom_pair1=[]
        atom_pair2=[]
        
        # select residues and calculate distances between them.
        atom_pair1=top.topology.select("resid "+str(start)+" and name CA")
        test=top.topology.select("resid 50 and name CA")
        atom_pair2=top.topology.select("resid "+str(end)+" and name CA")
        #print(atom_pair1,atom_pair2,test)
        atom_pairs=np.vstack([atom_pair1,atom_pair2])
        atom_pairs=atom_pairs.T
       
       
        #print(atom_pairs)
        #print(atom_sel.shape)
       
        ree=md.compute_distances(traj,atom_pairs)*10
       
        reeave=ree.mean()
        reestd=ree.std()
        #print(rgave)

       
        #plt.scatter(i+1,rgave)
        plt.errorbar(i+1,reeave,yerr=reestd,fmt='D',linewidth=2,capsize=3.5)
   
       
       

plt.legend(frameon=False,fontsize=16)

#plt.ylim([6,30])
plt.xlim([0,205])
       
plt.xlabel('Chain ID',fontsize=16)
plt.ylabel('EED (A)',fontsize=16)
plt.title(str(sim)+', LLPS vs. Single Chain, EED, '+str(m)+' K',fontsize=16)
plt.tick_params(axis='both', labelsize=16,direction='in',length=8,width=2)
plt.rcParams['axes.linewidth'] = 2
plt.rcParams["figure.figsize"] = (10,5)

### Coil-Helix transitions

In [ ]:
%cd /home/yumzhang/pikes_work/10-c36mrbdisp/
fig=plt.figure()


m=['aaqaa/ana/xr_ext_a99_2nd-rep/','aaqaa/ana/xp_mix_c36m_2nd-rep/','aaqaa-trex/ana/yz_mix_c36m_2nd-rep/',
  'aaqaa-rest3-scaleall/ana/yz_mix_c36m_rest3_2nd-rep/']
#m=['aaqaa-trex/ana/yz_mix_c36m_2nd-rep/']
dir='/home/yumzhang/pikes_work/10-c36mrbdisp/'
tempall=[298,308.461,319.289,330.497,342.098,354.107,366.537,379.404,392.722,406.507,420.777,435.548,450.837,466.662,483.044,500]
labels=['a99SB-disp+REST2 (1000 ns)','c36m+REST3 (3000 ns)','c36m+TREX (1500 ns)' ]


sim=1


markers=['o','D','*']
colors=['b','red','green','peru']
name=['a99sb-disp+REST3','c36mrbdisp+REST3','c36mrbdisp+TREX','c36m+REST3n']


inx=1   # plot
index=1 # Simulation results


for i in m:
    
    path=str(i)
    if index ==1:
        end=15
        temp=tempall[0:end]
    if index > 1:
        end=11
        temp=tempall[0:end]
        
    plt.tick_params(axis='both', labelsize=14,direction='in',length=6,pad=15)
    plt.rcParams['axes.linewidth'] = 2 
    plt.rcParams["figure.figsize"] = (6,5)
    print(index)
    
   # colors = plt.cm.rainbow(np.linspace(0, 1, (end+1)))
    allhctrans=np.arange(15)
    
    # for replicas
                            
    for j in range(0,end+1):
        plt.rcParams["figure.figsize"] = (6,5)
        
        dat1=path+str(j)+'-2nd-row.dat'
        if index==1:
            dat1=np.loadtxt(dat1)[10000:110000:100]
        if index>1:
            dat1=np.loadtxt(dat1)[200:1200] 
        if j == 1:
            print(dat1.shape,len(dat1[0]))
            
        reshc=[]
        for resl in range(len(dat1[0])):
            hindex=np.where(dat1[:,resl]==1)
            #print(hindex,np.diff(hindex),abs(np.diff(np.diff(hindex))))
            hctrans=abs(np.diff(np.diff(hindex)))
            hctrans=hctrans[hctrans>10]
            nhctrans=len(hctrans)
            #print(len(need))
            reshc=np.append(reshc,nhctrans)                
            
        reshc=reshc[1:16]

        allhctrans=np.vstack([allhctrans,reshc])
  
            
        plt.plot(np.arange(len(reshc))+1,reshc/2,c=colors[index-1],linewidth=0.5,linestyle=':')

    allhctrans=allhctrans[1:]/2
    allhctransave=np.mean(allhctrans,axis=0)
    allhctransstd=np.std(allhctrans,axis=0)
    plt.errorbar(np.arange(len(allhctransave))+1,allhctransave,
                 yerr=allhctransstd,c=colors[index-1],linewidth=2,label=name[index-1])
        
    index=index+1
    plt.legend()
    plt.xlim([1,15])
    

## Replica analysis

```
More detailed analysis can be found at:
/home/yumzhang/yz_analysis/rest-rest3-analysis/paper_svg_final
```

### Temperature diffusion

In [ ]:
# Take 8-rep as an example

%cd ~/Desktop/yz_pikes_work/10-c36mrbdisp/p53-rest3n/ana/
fig=plt.figure()

inx=1
sim=1
a=['star'] # marker style

colors=['blue','blue','green']
names=['REST3_8rep','REST3','8rep-REST3']
markers=['o','D','*']
color2=['cyan','darkorange','lime']

for i in a:
    path='yz_c36m_mix_temp_exchange/'
    plt.rcParams["figure.figsize"] = (36,12)
    
    for j in range(1,13):
        ax1 = plt.subplot(2, 6, inx)
        
        dat1=path+'aa'+str(j)+'-temp.dat' # dat: temperature walk as a function of time
        
        
        dat1=np.loadtxt(dat1)[::10] # select every 10 data point
        
        print(dat1.shape)
        
        dat1=dat1[0:510000]
        
        x=np.arange(len(dat1))
        x=x/50 # Change Time to ns
        
        temp=dat1[:,1]
        
        
        ax1.plot(x, dat1, color='grey')
        ax1.tick_params(axis='y', labelcolor='black',labelsize=20,direction='in')
        ax1.set_ylim([298,435])
        ax1.tick_params(axis='x', labelcolor='black',labelsize=22,direction='in')

        ## Below is for multiple Y-axis
        
        #ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        ##ax2.set_ylabel('Radius of Gyration (nm)',color=colors[sim],fontsize=16)  # we already handled the x-label with ax1
        #ax2.plot(x, rg , color=colors[sim],linewidth=1,linestyle='-')
        #ax2.axhline(y=2.38, color='black', linestyle='--',linewidth=1.5)
        #ax2.tick_params(axis='y', labelcolor=colors[sim],labelsize=14,direction='in')
        #ax2.set_ylim([0.8,5])
        #ax2.tick_params(axis='x', labelcolor='black',direction='in',labelsize=22)
        
        #ax2.spines["left"].set_position(("axes", -0.1))
        
        #ax3 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        #ax3.set_ylabel('End-to_End Distance')  # we already handled the x-label with ax1
        #ax3.plot(x, e2e , color='cyan',linewidth=1,linestyle=':')
        #ax3.tick_params(axis='y', labelcolor='black')
        #ax3.set_ylim([0,15])
        #ax3.spines["left"].set_position(("axes", -0.2))
        


        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        
        #plt.xlim([0,200])
        plt.tick_params(axis='both', labelsize=22,direction='in',length=6)
        plt.title(' Rep'+str(j),fontsize=22)
        plt.tight_layout()
        inx=inx+1
        
#plt.savefig('/home/yumzhang/pikes_work/4-rest/paper_fig/svg/p53_rest3_8rep_fold_random_walk_rg.svg')

### T0%

In [ ]:
inx=0
rep=16
colors=['blue','blue']
names=['REST3_fold','REST2_fold']
%cd /home/yumzhang/pikes_work/10-c36mrbdisp/p53/ana/yz_c36m_mix_temp_exchange




plt.rcParams["figure.figsize"] = (12,3)
plt.tick_params(axis='both', labelsize=16,direction='in',length=6)
# 16 rep, lim {0,0.15}; 8rep, lim {0,0.25}
plt.xticks(np.arange(1, rep+1, step=1))
plt.ylim([0,0.18])
plt.xlim([0,12.5])

for j in range(1,13):
        
        dat='aa'+str(j)+'-temp.dat'
        dat=np.loadtxt(dat)
        
        ### the previous REST2 traj have 5 more frequent save frequency
        
        #dat=dat[::5]
        
        print(dat.shape)
        # colume 1: frame, column 2: temp, exclude the 1st 200 ns 
        dat=dat[10000:,1]
        #print(dat.shape)
        # tt represents target temp, 298 K here!
        ntt=np.count_nonzero(dat <= 300)
        lowrate=ntt/len(dat)
        print(ntt,lowrate)
        
        plt.bar(j,lowrate,color=colors[inx],width=0.4)
        # y is average 1/rep
        plt.axhline(y=1/rep, color='grey', linestyle=':',linewidth=1.5)
        inx=inx+1
        
plt.ylim([0,0.5])       
plt.ylabel('298K %',fontsize=16)
plt.xlabel('Replicas',fontsize=16)
plt.tick_params(axis='both', labelsize=16,direction='in',length=6)
plt.rcParams['axes.linewidth'] = 2

### <T>

In [ ]:
inx=0

rep=12
colors=['blue','blue']
#names=['REST3_ctrl','REST2_fold']
ave=360

path='temp-exchange/'
plt.rcParams["figure.figsize"] = (12,3)
plt.tick_params(axis='both', labelsize=16,direction='in',length=6)
plt.xticks(np.arange(1, rep+1, step=1))
plt.ylim([298,450])
plt.xlim([0,rep+1])
    
for j in range(1,13):
        
        dat='aa'+str(j)+'-temp.dat'
        ### the previous REST2 traj have 5 more frequent save frequency
        
        dat=np.loadtxt(dat)
        # colume 1: frame, column 2: temp, exclude frist 200 ns
        dat=dat[10000:,1]
        
        # tt represents target temp, 298 K here!
        tave=np.mean(dat)
        
        print(tave)
        
        plt.bar(j,tave,color=colors[inx],width=0.4)
        plt.axhline(y=ave, color='grey', linestyle=':',linewidth=1.5)

    #plt.savefig('/home/yumzhang/pikes_work/4-rest/paper_fig/svg_update/fig3_s3.kid_'+names[inx]+'_Tave_rep-update.svg',dpi=300)
        inx=inx+1
plt.ylabel('<T>',fontsize=16)
plt.xlabel('Replicas',fontsize=16)

### Temperature round travels

In [ ]:
a=['p53']
# remember to change j range! modify all scripts next time
rep=12
nrep=12
low=300
high=430

colors=['blue','blue','blue','blue']
names=['REST2_ctrl','REST2_fold','REST3_ctrl','RESt3_fold']


nt=0
for i in a:
    path='temp-exchange/'
    plt.rcParams["figure.figsize"] = (12,3)
    plt.tick_params(axis='both', labelsize=16,direction='in',length=6)
    print(i)
    plt.xticks(np.arange(1, rep+1, step=1))
    plt.ylim([0,50])
    plt.xlim([0,rep+1])

    
    # j range: replica
    for j in range(1,13):
        
        dat='aa'+str(j)+'-temp.dat'
        ### the previous REST2 traj have 5 more frequent save frequency
       
        dat=np.loadtxt(dat)#[::5]
        print(dat.shape)
        
        # colume 1: frame, column 2: temp
        dat=dat[0:,1]
        print(dat.shape)
        
        
        count=0
        
        ## m: starting data point
        ## exclude the first 200 ns
        m=10000
        
        ## loop until frame 510000
        while m <= 510000:
            tnow=dat[m]
            plus=1
            
            ### add break, or m=m+1, can plus to infinite!
            if tnow < high:
                m=m+1
                if m>len(dat)-1:
                    break
                else:
                    tnow=dat[m]
                #print(m, 'tnow<high',tnow)
            else: 
                m=m+1
                if m>len(dat)-1:
                    break
                else:
                    tnext=dat[m]
                #print(m, 'tnow>high, to tnext:',tnext)
                
                while tnext > low:
                    m=m+1
                    if m>len(dat)-1:
                        break
                    else:
                        tnext=dat[m]
                    #print(m, 'tnext still > low:',tnext)
                else:
                    #print('Here is a rt: count=',count)
                    count=count+plus
                    plus=0
                    m=m+1
                    if m>len(dat)-1:
                        break
        count=count
        plt.bar(j,count,width=0.4,color='b')
        nt=nt+count
    avent=nt/nrep
    print(avent)
    plt.axhline(y=avent, color='grey', linestyle=':',linewidth=1.5)
    plt.ylim([0,45])  
    plt.tick_params(axis='both', labelsize=22,direction='in',length=6)
   
            
    plt.ylabel('Ntrans/us',fontsize=16)
    plt.xlabel('Replicas',fontsize=16)

## More sophiscated analysis

### 2D-plot for RG+REE

In [ ]:
## Heatmap
## cubehelix
def plot_heatmap(ax,start,end):
    ## this is a histogram of the number of occurrences of the observations at (x[i],y[i]).
    ## bin: 'log', use a logarithmic scale for the color map. 
    ## Internally, log10(i+1) is used to determine the hexagon color. 
    ## vmin and vmax for colorbar range
    hb=ax.hexbin(x, y,
              cmap=sns.cubehelix_palette(dark=0,light=1,as_cmap=True),
              mincnt=1,
              bins='log',
              gridsize=50,
              vmin=0,
              vmax=2.6
              )
    # show color bar
    cb = fig.colorbar(hb, ax=ax)
    ax.set_xlabel("PC1", fontsize=16)
    ax.set_ylabel("PC2", fontsize=16)
## 
def plot_2dhist(ax,start,end):
    #range is decided by Rg and EED range
    plt.hist2d(x, y,bins=(50, 50), range=np.array([(10, 45), (0, 130)]),cmap=plt.cm.jet, normed=True)
    # show color bar
    cb = plt.colorbar()
    plt.clim(0,1.2)
    ax.tick_params(labelsize=30)
    cb.ax.tick_params(labelsize=25)
    ax.set_xlabel("PC1", fontsize=16)
    ax.set_ylabel("PC2", fontsize=16)
##
def plot_2dhist_v2(ax,start,end):
    h = hist2d.Hist2D(df)
    h.plot(bins=[50,50], cmap='hot_r', contour=False, normed=True, range=[[10,45],[0,130]],grid=False, colorbar=False, xlabel='', ylabel='')
    cb = plt.colorbar()
    plt.clim(0,1.2)
    ax.tick_params(labelsize=0)

##-------------Load data and plot------------------
##=============

namelist=["HyRes", "HyRes II"]#,"HyRes II, s1","HyRes II, s2"] #, "d49y_f", "w53g_f"]
framelst=[0,80000,160000] #,120001,160000] #291221, 170032,   177390,   368855,   246110]
plt.rcParams['axes.linewidth'] = 2
#fig, ax = plt.subplots(figsize=(6, 5))
for i in range(0,2):
    start=framelst[i]
    end=framelst[i+1]
    x=dat[start:end,0]
    y=dat[start:end,1]
    fname=namelist[i]
    H, xedges, yedges = np.histogram2d(x,y, bins=[50, 50], range=[[10,45],[0,130]],density=True)

    # plot 2d histogram in -log scale
    fig, ax = plt.subplots(figsize=(6, 5))
    # shift minimum value to 0
    H_min=min(-1*np.log(H.T)[np.isfinite(np.log(H.T))])
    Z=-1*np.log(H.T)-H_min
    # heatmap
    extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.imshow(Z, aspect='auto', origin='lower', extent=extent, cmap=plt.cm.jet)
    # contour
    levels = np.arange(0, 8, 1)
    ax.contour(Z, levels, colors='k', origin='lower', extent=extent)
    # colorbar
    cb = plt.colorbar()
    plt.clim(0,8)
    #plt.title(fname+', p53, Ca-coordinates',fontsize=16)
    #plt.legend(fontsize=14)
    #plt.xlabel('PC 1',fontsize=16)
    #plt.ylabel('PC 2',fontsize=16)
    fig.tight_layout()
    plt.tick_params(axis='both', labelsize=16,direction='in',length=6)
    cb.ax.tick_params(labelsize=16)

### Partial Helicity calcultions + plot

In [13]:
# %load /Users/muk/Desktop/yz_pikes_work/4-rest/ana/script/s9-partial-helix.py
# this is for calculations

#!/home/xrliu/program/anaconda/anaconda3/bin/python
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt

inx=1

#fig=plt.figure()
#x=np.loadtxt('rest3-2nd-all.dat')

# better to give an abolute path!
A='../1kdx_at_ext-rest2/2nd-cond/'
B='../1kdx_at_ext-2rest3/2nd-cond/'
C='../1kdx_at_ext-8rep-rest2/2nd-cond/'
D='../1kdx_at-8rep-rest2/2nd-cond/'
E='../1kdx_at-rest2/2nd-cond/'
F='../1kdx_at-2rest3/2nd-cond/'

H='../p53_at_ext-rest2/2nd-cond/'
I='../p53_at_ext-2rest3/2nd-cond/'
G='../p53_at_ext-8rep-2rest3/2nd-cond/'
Q=[H,I]
## for kid, 0-2nd-row, for p53, 0-row.dat
for q in Q:
    x=np.loadtxt(str(q)+'0-row.dat')
    print(x.shape)
    
    # partial helicity as a function of time (Can make time a variable!)
    t1=x[0:250]
    t2=x[0:500]
    t3=x[0:750]
    t4=x[0:1000]
    t5=x[0:2000]
    t6=x[0:4000]
    t7=x[0:6000]
    t8=x[0:8000]
    t9=x[0:10000]
    t10=x[0:15000]
    t11=x[0:20000]


    time=[t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11]
    #time=[t10,t11]
    for m in time:
        h=np.where(m==1)
        data=h[1]
        print(str(q),data.shape)
        count=0
        with open(str(q)+str(len(m))+'-res-len.dat', 'w') as f:
            for j in range(len(data)-1):
                x=data[j]
                y=data[j+1]
                a=(y-x)
                if a==1:
                    j=j+1                    
                    count=count+1
                else:
                    res=data[j-count]
                    length=count+1            
                    print(res,length,file=f)                
                    j=j+1
                    count=0
     



In [14]:
# This is for plot 
## Heatmap
## cubehelix
def plot_heatmap(ax,start,end):
    ## this is a histogram of the number of occurrences of the observations at (x[i],y[i]).
    ## bin: 'log', use a logarithmic scale for the color map. 
    ## Internally, log10(i+1) is used to determine the hexagon color. 
    ## vmin and vmax for colorbar range
    hb=ax.hexbin(x, y,
              cmap=sns.cubehelix_palette(dark=0,light=1,as_cmap=True),
              mincnt=1,
              bins='log',
              gridsize=50,
              vmin=0,
              vmax=2.6
              )
    # show color bar
    cb = fig.colorbar(hb, ax=ax)
    ax.set_xlabel("PC1", fontsize=16)
    ax.set_ylabel("PC2", fontsize=16)
## 
def plot_2dhist(ax,start,end):
    #range is decided by Rg and EED range
    plt.hist2d(x, y,bins=(50, 50), range=np.array([(0, 61), (2, 15)]),cmap=plt.cm.jet, normed=True)
    # show color bar
    cb = plt.colorbar()
    plt.clim(0,0.8)
    ax.tick_params(labelsize=30)
    cb.ax.tick_params(labelsize=25)
    ax.set_xlabel("PC1", fontsize=16)
    ax.set_ylabel("PC2", fontsize=16)
##
def plot_2dhist_v2(ax,start,end):
    h = hist2d.Hist2D(df)
    h.plot(bins=[50,50], cmap='hot_r', contour=False, normed=True, range=[[0,61],[2,15]],grid=False, colorbar=False, xlabel='', ylabel='')
    cb = plt.colorbar()
    plt.clim(0,0.8)
    ax.tick_params(labelsize=0)


# Saved data as a function of time
namelist=["25","50","75","100", "200", "400","600","800","1000"]#,'1500','2000']#,"1200"]#,"1400"]#"1500","2000","2500","3000"] #, "d49y_f", "w53g_f"]
framelst=["250","500","750","1000","2000","4000","6000","8000","10000"]#,"15000","20000"]#,"12000"]#,"14000"]#,"25000","30000"] #291221, 170032,   177390,   368855,   246110]

shift=0
start=0
inx=1
fig, ax = plt.subplots(figsize=(18, 15))
for i in range(0,9):
    plt.subplot(3,3,inx)

    txx1=np.loadtxt('1kdx_at-2rest3/2nd-cond/'+framelst[i]+'-res-len.dat')
    txx2=np.loadtxt('1kdx_at_ext-2rest3/2nd-cond/'+framelst[i]+'-res-len.dat')

    x1=txx1[:,0]
    y1=txx1[:,1]
    x2=txx2[:,0]
    y2=txx2[:,1]
    
    x=np.concatenate([x1,x2],axis=0)
    y=np.concatenate([y1,y2],axis=0)
   
    fname=namelist[i]
    H, xedges, yedges = np.histogram2d(x,y, bins=[10, 10], range=[[0,28],[2,15]],density=True)

    # plot 2d histogram in -log scale
    #fig, ax = plt.subplots(figsize=(6, 5))
    # shift minimum value to 0
    H_min=min(-1*np.log(H.T)[np.isfinite(np.log(H.T))])
    Z=-1*np.log(H.T)-H_min
    # heatmap
    extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.imshow(Z, aspect='auto', origin='lower', extent=extent, cmap=plt.cm.jet)
    # contour
    levels = np.arange(0, 3, 1)
    #ax.contour(Z, levels, colors='k', origin='lower', extent=extent)
    # colorbar
    cb = plt.colorbar()
    plt.clim(0,3)
    plt.text(1,13,fname+' ns',fontsize=45)
    #plt.legend(fontsize=14)
    #plt.
    #plt.xlabel('Residues',fontsize=16)
    #plt.ylabel('Length',fontsize=16)
    fig.tight_layout()
    plt.tick_params(axis='both', labelsize=30,direction='in',length=6)
    plt.rcParams['axes.linewidth'] = 4
    cb.ax.tick_params(labelsize=30)
    
    inx=inx+1
plt.savefig('/home/yumzhang/pikes_work/4-rest/paper_fig/svg_update/figs11.kid-rest3-PL-all.svg')